In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

df_1 = pd.read_csv("../output/task2-1.csv")
df_2 = pd.read_csv("../output/task2-2.csv")
df_2_total = pd.read_csv("../output/task2-2-total.csv")

In [2]:
# plot the number of cells over time
plt.figure(figsize=(8, 3))
plt.plot(df_1['time'], df_1['cells'], label='Cells')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-1.pdf', bbox_inches='tight')
plt.close()

In [3]:
plt.figure(figsize=(3, 3))
plt.plot(df_2['x'], df_2['y'], label='_task2-2', color='gray')
plt.plot(df_2['x'][0], df_2['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_2['x'].iloc[-1], df_2['y'].iloc[-1], 'rx', label='End', ms=10, mew=2)
plt.xlim(25,75)
plt.ylim(25,75)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig('../../report/graphics/task2-2.pdf', bbox_inches='tight')
plt.close()

In [4]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-m"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-m/{file}")
        # log to base 10 for power
        record["m"] = int(name[8:])
        data[name] = record

for name, record in data.items():
    dt = 0.001
    record['df']['time'] = record['df']['time'] * dt

In [5]:
# plot the number of cells over time for all m
plt.figure(figsize=(8, 4))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=('$10^{' + str(record["m"]) + '}$'))
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Capacity')
plt.yscale('log')
plt.ylim(1e9, 1e17)
plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')

In [6]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-h"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-h/{file}")
        # log to base 10 for power
        record["m"] = int(name[10:])
        data[name] = record

# scale time
for name, record in data.items():
    dt = 10 ** record['m']
    record['df']['time'] = record['df']['time'] * dt

In [7]:
# solve analytically first
x = np.linspace(0, 1200, 2000)
y = 10 ** (13 - (4 * (np.e ** (-0.006 * x))))
analytical = pd.DataFrame({'time': x, 'cells': y})

# plot the number of cells over time for all h
plt.figure(figsize=(8, 4))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=('$10^{' + str(record["m"]) + '}$'))
plt.plot(analytical['time'], analytical['cells'], label='Analytical')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Dt')
plt.savefig('../../report/graphics/task2-1-h.pdf', bbox_inches='tight')

In [8]:
# plot again but zoomed in ar 450-550
plt.figure(figsize=(8, 4))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=('$10^{' + str(record["m"]) + '}$'))
plt.plot(analytical['time'], analytical['cells'], label='Analytical')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Dt')
plt.xlim(450, 550)
plt.ylim(0.55e13, 0.65e13)
plt.savefig('../../report/graphics/task2-1-h-zoom.pdf', bbox_inches='tight')
plt.close()

In [9]:
# plot the number of cells over time
plt.figure(figsize=(8, 4))
plt.plot(df_2_total['time'], df_2_total['cells'], label='Cells', color='gray')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-2-total.pdf', bbox_inches='tight')

In [10]:
# plot the number of cells over time
plt.figure(figsize=(8, 4))
plt.plot(df_2_total['time'], df_2_total['cells'], label='Cells', color='gray')


# estimate a linear fit
x = df_2_total['time']
y = df_2_total['cells']

A = np.vstack([x, np.ones(len(x))]).T
m, c = np.linalg.lstsq(A, y, rcond=None)[0]
print(f"m={m}, c={c}")

# dashed
plt.plot(x, m*x + c, 'r', label='Fitted line', linestyle='dashed')

# add text with the equation
plt.text(0, 0.9e15, f'y = {m:.4g}x + {c:.2g}', fontsize=12)

plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-2-total-linear.pdf', bbox_inches='tight')

m=8290757.139360582, c=0.10363446464054554
